In [9]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

In [10]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [3]:
#Adicione o caminho do seu arquivo aqui.
file = "/content/drive/My Drive/Colab Notebooks/USA_Housing.csv"
input_df = pd.read_csv(file)

In [4]:
from sklearn.preprocessing import LabelEncoder
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [5]:
class MachineLearningPipeline:
  def __init__(self,df):
    self.df = df
  
  def dataCleaning(self):
    self.df.dropna()

    
  def selectColumns(self):
    self.X = self.df[['Avg. Area Income', 'Avg. Area House Age',\
                     'Avg. Area Number of Rooms', 'Avg. Area Number of Bedrooms', \
                     'Area Population']].values
    self.y = self.df['Price'].values

  def selectColumnsDF(self):
    self.df['Avg. Area Income'] = np.log(self.df['Avg. Area Income'])
    self.df['Avg. Area House Age'] = np.log(self.df['Avg. Area House Age'])
    self.df['Avg. Area Number of Rooms']  = np.log(self.df['Avg. Area Number of Rooms'])
    self.df['Avg. Area Number of Bedrooms']  = np.log(self.df['Avg. Area Number of Bedrooms'])
    self.df['Area Population']  = np.log(self.df['Area Population'])
    self.df = self.df[['Avg. Area Income', 'Avg. Area House Age',\
                     'Avg. Area Number of Rooms', 'Avg. Area Number of Bedrooms', \
                     'Area Population','Price']].values
  

  def selectColumn(self):
    self.X = self.df['Avg. Area Income'].values
    self.y = self.df['Price'].values
    
  
  def trainTestSplit(self):
    self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size=0.2, random_state=0)
  

In [6]:
pipeline = MachineLearningPipeline(input_df)
pipeline.dataCleaning()
pipeline.selectColumn()
pipeline.trainTestSplit()

In [7]:
def mean_absolute_error(y, y_pred):
  N = len(y)
  error = (1/N) * np.sum(abs(y-y_pred))
  return error

def mean_squared_error(y, y_pred):
  N = len(y)
  error = (1/N) * (np.sum(np.array(y-y_pred))** 2)
  return error

def root_mean_squared_error(y, y_pred):
  N = len(y)
  error = np.sqrt((1/N) * np.sum((y-y_pred)** 2))
  return error

def r_squared(y, y_pred):
  yhat = y_pred                    
  ybar = np.sum(y)/len(y)          
  ssreg = np.sum((yhat-ybar)**2)
  sstot = np.sum((y - ybar)**2)
  error = 1 - (ssreg / sstot)
  return error

# Perceptron & Backpropagation

https://medium.com/@a.mirzaei69/implement-a-neural-network-from-scratch-with-python-numpy-backpropagation-e82b70caa9bb

In [1]:
import numpy as np
class NeuralNetwork(object):
  def __init__(self, layers = [2 , 10, 1], activations=['sigmoid', 'sigmoid']):
      assert(len(layers) == len(activations)+1)
      self.layers = layers
      self.activations = activations
      self.weights = []
      self.biases = []
      for i in range(len(layers)-1):
          self.weights.append(np.random.randn(layers[i+1], layers[i]))
          self.biases.append(np.random.randn(layers[i+1], 1))

In [3]:
  def feedforward(self, x):
      # return the feedforward value for x
      a = np.copy(x)
      z_s = []
      a_s = [a]
      for i in range(len(self.weights)):
          activation_function = self.getActivationFunction(self.activations[i])
          z_s.append(self.weights[i].dot(a) + self.biases[i])
          a = activation_function(z_s[-1])
          a_s.append(a)
      return (z_s, a_s)

In [4]:
  def backpropagation(self,y, z_s, a_s):
        dw = []  # dC/dW
        db = []  # dC/dB
        deltas = [None] * len(self.weights)  # delta = dC/dZ  known as error for each layer
        # insert the last layer error
        deltas[-1] = ((y-a_s[-1])*(self.getDerivitiveActivationFunction(self.activations[-1]))(z_s[-1]))
        # Perform BackPropagation
        for i in reversed(range(len(deltas)-1)):
            deltas[i] = self.weights[i+1].T.dot(deltas[i+1])*(self.getDerivitiveActivationFunction(self.activations[i])(z_s[i]))        
        #a= [print(d.shape) for d in deltas]
        batch_size = y.shape[1]
        db = [d.dot(np.ones((batch_size,1)))/float(batch_size) for d in deltas]
        dw = [d.dot(a_s[i].T)/float(batch_size) for i,d in enumerate(deltas)]
        # return the derivitives respect to weight matrix and biases
        return dw, db

In [5]:
  def train(self, x, y, batch_size=10, epochs=100, lr = 0.01):
  # update weights and biases based on the output
      for e in range(epochs): 
          i=0
          while(i<len(y)):
              x_batch = x[i:i+batch_size]
              y_batch = y[i:i+batch_size]
              i = i+batch_size
              z_s, a_s = self.feedforward(x_batch)
              dw, db = self.backpropagation(y_batch, z_s, a_s)
              self.weights = [w+lr*dweight for w,dweight in  zip(self.weights, dw)]
              self.biases = [w+lr*dbias for w,dbias in  zip(self.biases, db)]
              print("loss = {}".format(np.linalg.norm(a_s[-1]-y_batch) ))


In [6]:
  @staticmethod
  def getActivationFunction(name):
      if(name == 'sigmoid'):
          return lambda x : np.exp(x)/(1+np.exp(x))
      elif(name == 'linear'):
          return lambda x : x
      elif(name == 'relu'):
          def relu(x):
              y = np.copy(x)
              y[y<0] = 0
              return y
          return relu
      else:
          print('Unknown activation function. linear is used')
          return lambda x: x
    


In [7]:
  @staticmethod
  def getDerivitiveActivationFunction(name):
      if(name == 'sigmoid'):
          sig = lambda x : np.exp(x)/(1+np.exp(x))
          return lambda x :sig(x)*(1-sig(x)) 
      elif(name == 'linear'):
          return lambda x: 1
      elif(name == 'relu'):
          def relu_diff(x):
              y = np.copy(x)
              y[y>=0] = 1
              y[y<0] = 0
              return y
          return relu_diff
      else:
          print('Unknown activation function. linear is used')
          return lambda x: 1

In [8]:
if __name__=='__main__':
    import matplotlib.pyplot as plt
    nn = NeuralNetwork([1, 100, 1],activations=['sigmoid', 'sigmoid'])
    X = np.log(pipeline.X).reshape(1, -1)
    y = np.log(pipeline.y).reshape(1, -1)
    
    nn.train(X, y, epochs=100, batch_size=32, lr = .001)
    _, a_s = nn.feedforward(X)

    print(mean_absolute_error(y.flatten(), a_s[-1].flatten()))

NameError: name 'pipeline' is not defined

# CNN


In [9]:
import keras
keras.__version__

'2.6.0'

In [10]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.layers import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.preprocessing import image


In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
classificador = Sequential()

In [13]:
classificador.add(Conv2D(32, (3,3), input_shape = (64, 64, 3), activation = 'relu'))
classificador.add(BatchNormalization())
classificador.add(MaxPooling2D(pool_size = (2,2)))
classificador.add(Conv2D(32, (3,3), input_shape = (64, 64, 3), activation = 'relu'))
classificador.add(BatchNormalization())
classificador.add(MaxPooling2D(pool_size = (2,2)))

classificador.add(Flatten())

classificador.add(Dense(units = 128, activation = 'relu'))
classificador.add(Dropout(0.2))
classificador.add(Dense(units = 128, activation = 'relu'))
classificador.add(Dropout(0.2))
classificador.add(Dense(units = 1, activation = 'sigmoid'))

In [14]:
print(classificador.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 62, 62, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 29, 29, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0

In [15]:
classificador.compile(optimizer = 'adam', loss = 'binary_crossentropy',
                      metrics = ['accuracy'])


In [16]:
gerador_treinamento = ImageDataGenerator(rescale = 1./255,
                                         rotation_range = 7,
                                         horizontal_flip = True,
                                         shear_range = 0.2,
                                         height_shift_range = 0.07,
                                         zoom_range = 0.2)

In [17]:
gerador_teste = ImageDataGenerator(rescale = 1./255)

In [18]:
file = "/content/drive/My Drive/Colab Notebooks/dataset/training_set"

base_treinamento = gerador_treinamento.flow_from_directory(file,
                                                           target_size = (64, 64),
                                                           batch_size = 32,
                                                           class_mode = 'binary')

Found 4000 images belonging to 2 classes.


In [19]:
file = "/content/drive/My Drive/Colab Notebooks/dataset/test_set"

base_teste = gerador_teste.flow_from_directory(file,
                                               target_size = (64, 64),
                                               batch_size = 32,
                                               class_mode = 'binary')

Found 1000 images belonging to 2 classes.


In [ ]:
classificador.fit_generator(base_treinamento, steps_per_epoch = 4000 / 32,
                            epochs = 40, validation_data = base_teste,
                            validation_steps = 1000 / 32)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
 40/125 [========>.....................] - ETA: 14:53 - loss: 0.8914 - accuracy: 0.5625

In [ ]:
imagem_teste = image.load_img('/content/drive/My Drive/Colab Notebooks/dataset/test_set/cachorro/dog.3526.jpg',
                              target_size = (64,64))
imagem_teste = image.img_to_array(imagem_teste)
imagem_teste /= 255
imagem_teste = np.expand_dims(imagem_teste, axis = 0)
previsao = classificador.predict(imagem_teste)

previsao = (previsao > 0.5)

previsao